In [1]:
import warnings
from pprint import pprint

warnings.filterwarnings("ignore", category=DeprecationWarning) 
!warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
import torch

Распределите в команде гпу, задайте свой номер.

In [6]:
import os
# Set proper device for computations,
dtype, device, cuda_device_id = torch.float32, None, 3
os.environ["CUDA_VISIBLE_DEVICES"] = '{0}'.format(str(cuda_device_id) if cuda_device_id is not None else '')
if cuda_device_id is not None and torch.cuda.is_available():
    device = 'cuda:{0:d}'.format(0)
else:
    device = torch.device('cpu')
print(f'dtype: {dtype}, device: {device}, cuda_device_id {cuda_device_id}')

dtype: torch.float32, device: cuda:0, cuda_device_id 3


Импортируем полезную либу `attrdict`. Чем она хороша: позволяет обращаться к элементам словаря, как к его атрибутам

In [7]:
from attrdict import AttrDict

# Работа с аудио и текстом

В распознавании речи нейронная сеть обучается на парах аудио+текст.

Давайте научимся открывать аудиофайлы и подготавливать их для работы с нейронной сетью.



На лекциях мы обсуждали, что аудио может быть записано с разной частотой дискретизации (sample rate), но для обучения нейронной сети обычно все аудио приводят к одной частоте дискретизации. (В этом проекте мы будем использовать sample rate 8000).

Так же аудио может быть одноканальным или многоканальным. Проблема с многоканальным аудио может быть в том, что речь в двух каналах может звучать одновременно, такие аудио надо разделять на 2 дорожки. В нашем же случае данные подобраны таким образом, что такой ситуации не возникет, поэтому несколько каналов можно просто усреднить.


Будем использовать библиотеку `torchaudio` [docs](https://pytorch.org/audio/).

Реализуйте функцию `open_audio`, которая открывает аудио (искать [тут](https://pytorch.org/audio/stable/torchaudio.html)), усредняет аудио по всем каналам (это можно сделать обычным усреднением) и приводит к необходимой частоте дискретизации (искать [тут](https://pytorch.org/audio/stable/transforms.html)).

In [8]:
import torchaudio

In [9]:
def open_audio(audio_path, desired_sample_rate):
    
    """ Open and resample audio, average across channels
        Inputs:
            audio_path: str, path to audio
            desired_sample_rate: int, the sampling rate to which we would like to convert the audio
        Returns:
            audio: 1D tensor with shape (num_timesteps)
            audio_len: int, len of audio
    """
    waveform, sample_rate = torchaudio.load(audio_path)
    resampling = torchaudio.transforms.Resample(
        sample_rate,
        desired_sample_rate,
        resampling_method='sinc_interpolation',
    )
    audio = resampling(waveform)
    audio = torch.transpose(audio, 0, 1) #audio = audio.transpose(0, 1)
    audio = torch.mean(audio, 1) #audio = audio.mean(dim=1)
    audio_len = audio.shape[0]
    
    return audio, audio_len


Запустите тесты, чтобы проверить себя.
<img src="images/tests_are_all_we_need.png" width="400" height="600">


**Перед каждым запуском тестов не забывайте сохранять ноутбук.**

In [ ]:
! pytest tests/test_open_audio.py

Теперь давайте откроем аудио, и послушаем, что у нас получилось.

In [10]:
from IPython.display import Audio

In [11]:
sample_rate=8000
audio, audio_len = open_audio('test_files/test_audio.mp3', sample_rate)
Audio(data=audio.numpy(), rate=sample_rate)

Так же послушать аудио можно через путь к аудио файлу.

Можно заметить, что звучание немного поменялось. Это произошло из-за того, что мы поменяли оригинальный sample rate 48000Hz на 8000Hz.

In [12]:
Audio('test_files/test_audio.mp3')

Speech2text  —  это не только speech, но и text, поэтому теперь давайте поговорим о предобработке текста. 

Первым шагом необходимо привести текст к нижнему регистру и удалить пунктуацию. Если на этом этапе в тексте содержатся символы кроме русского алфавита и пробела (например, цифры), то такие примеры лучше вообще убрать из обучающей выборки (если просто убрать символ из строки, то может нарушиться соответствие аудио-текст).

В размеченных данных для обучения и валидации уже произедена очистка и удаление ненужных символов, поэтому нам надо только извлечь токены из текста.

В качестве токенов для обучения нейронной сети будем использовать буквы русского алфавита и пробел. Так же, как было сказано в лекциях, нам потребуется специальный символ `<blank>` для построения выравниваний.

In [13]:
alphabet = ['а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к',
            'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц',
            'ч', 'ш', 'щ', 'ь', 'ы', 'ъ', 'э', 'ю', 'я',
            ' ', '<blank>']

Но, естественно, в таком виде передавать буквы в нейронную сеть мы не можем — их нужно закодировать в числа. Для этого будем использовать уже готовую функцию Vocab.

Обратите внимание, что Vocab добавляет дополнительный токен `<unk>`, ответственный за все символы, которых нет в словаре.

In [14]:
from vocabulary import Vocab
vocab = Vocab(alphabet)

The `unk_token` '<unk>' wasn't found in the tokens. Adding the `unk_token` to the end of the Vocab.


In [15]:
vocab

Vocab()

Попрактикуйтесь с Vocab. Для того, чтобы понять все возможности Vocab можно заглянуть в код vocabulary.py и почитать докстринги.

* переведите "привет" в индексы
* переведите [11, 0, 11, 33, 4, 5, 12, 0] в текст
* что будет если перевести в индексы слово "hi"?
* а [44, 5] в текст?


In [ ]:
string = "привет"
vocab.lookup_indices(string)

In [ ]:
vocab.lookup_tokens([11, 0, 11, 33, 4, 5, 12, 0])

In [ ]:
string = "hi"
vocab.lookup_indices(string)

Какие еще у Vocab возможности?

Если в процессе исследования вы изменили текущий vocab (путем добавления нового токена, например), не забудьте вернуть vocab к начальному состоянию  `vocab = Vocab(alphabet)`

Давайте извлечем из Vocab и сохраним в переменные два важных значения - длину алфавита и значения индекса `<blank>`.
    
Эти значения нам еще много раз пригодятся.

In [16]:
def get_num_tokens(vocab):
    num_tokens = len(vocab)
    return num_tokens

def get_blank_index(vocab):
    blank_index = vocab['<blank>']
    return blank_index

In [17]:
! pytest tests/test_num_tokens.py
! pytest tests/test_blank_index.py

Don't forget to save Jupyter Notebook! 


============================= test session starts ==============================
platform linux -- Python 3.6.9, pytest-6.2.2, py-1.10.0, pluggy-0.13.1
rootdir: /home/olya/sirius-stt, configfile: pytest.ini
collected 1 item                                                               

tests/test_num_tokens.py .                                               [100%]

============================== 1 passed in 1.84s ===============================
Don't forget to save Jupyter Notebook! 


============================= test session starts ==============================
platform linux -- Python 3.6.9, pytest-6.2.2, py-1.10.0, pluggy-0.13.1
rootdir: /home/olya/sirius-stt, configfile: pytest.ini
collected 1 item                                                               

tests/test_blank_index.py .                                              [100%]

============================== 1 passed in 1.83s ===============================


In [18]:
num_tokens = get_num_tokens(vocab) 
blank_index = get_blank_index(vocab)

# Подготовка датасета

Обучать нейронную сеть мы будем на подготовленном датасете. Для удобства мы сгруппировали обучающие и валидационные данные в текстовые файлы, где в каждой строке через запятую указан путь к файлу, соответствующий текст, длина аудио (в секундах).


Давайте реализуем функцию AudioDataset, которая подготавливает текст и аудио для каждого элемента датастета.


На этих данных мы будем обучать модель с помощью ctc лосса, а мы уже знаем, что для обучения с ctc аудио лучше отсортировать по длине - так модели будет проще обучиться.

Изучить документацию по Dataset и DataLoader (пригодится далее) можно найти [тут](https://pytorch.org/docs/stable/data.html).

In [19]:
import pandas as pd

In [20]:
 class AudioDataset(torch.utils.data.Dataset):
    def __init__(self,
                 dataset_path,
                 vocab,
                 sample_rate=8000,
                 ):
        self.vocab = vocab
        self.sample_rate = sample_rate
        data = pd.read_csv(
            dataset_path,
            header=None,
            names=['audio_path', 'text', 'duration'],
        )
        data['duration'] = data['duration'].astype(float)
        self.data = data.sort_values(by='duration')

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        audio_path = self.data.audio_path[idx]
        text = self.data.text[idx]
        text_len = len(text)
        audio, audio_len = open_audio(audio_path, self.sample_rate)
        tokens = torch.tensor(self.vocab.lookup_indices(text), dtype=torch.long)
        
        return {"audio":  audio,  # torch tensor, (num_timesteps)
                "audio_len": audio_len, # int
                "text": text, # str
                "text_len": text_len, # int
                "tokens": tokens, # torch tensor, (text_len)
               }

In [21]:
! pytest tests/test_dataset.py

Don't forget to save Jupyter Notebook! 


============================= test session starts ==============================
platform linux -- Python 3.6.9, pytest-6.2.2, py-1.10.0, pluggy-0.13.1
rootdir: /home/olya/sirius-stt, configfile: pytest.ini
collected 2 items                                                              

tests/test_dataset.py ..                                                 [100%]

============================== 2 passed in 2.01s ===============================


Давайте посмотрим, как будут выглядеть элементы полученного датасета.

In [22]:
train_dataset = '/home/e.chuykova/data/train.txt'
val_dataset = '/home/e.chuykova/data/val.txt'
test_dataset = '/home/e.chuykova/data/test.txt'

In [23]:
dataset = AudioDataset(train_dataset, vocab)

for i, element in enumerate(dataset):
    print(f'element number: {i}')
    print(element)
    print()
    if i > 1:
        break

element number: 0
{'audio': tensor([ 0.0000,  0.0000,  0.0000,  ...,  0.0005, -0.0008, -0.0006]), 'audio_len': 8360, 'text': 'один', 'text_len': 4, 'tokens': tensor([15,  4,  9, 14])}

element number: 1
{'audio': tensor([-1.3758e-09, -2.2855e-09, -1.3640e-09,  ..., -2.1114e-04,
        -4.1249e-04, -2.6402e-04]), 'audio_len': 8640, 'text': 'ноль', 'text_len': 4, 'tokens': tensor([14, 15, 12, 27])}

element number: 2
{'audio': tensor([0.0000, 0.0000, 0.0000,  ..., 0.0068, 0.0080, 0.0038]), 'audio_len': 8987, 'text': 'два', 'text_len': 3, 'tokens': tensor([4, 2, 0])}



Чтобы эффективно обучать нейронную сеть, необходимо подавать в нее данные батчами. В этом нам поможет функция `torch.utils.data.DataLoader`. 

Обратите внимание, что некоторые данные в датасете разной длины (например, `audio`), для формирования батча из таких данных  необходимо использовать паддинг. Для этого можно реализовать фукцию `collate_fn`. Подробнее про то, как именно использовать `collate_fn` можно почитатать в доках к `torch.utils.data.DataLoader`.

In [24]:
from torch.nn.utils.rnn import pad_sequence

In [25]:
def collate_fn(batch):
    """ 
        Inputs:
            batch: list of elements with length=batch_ize
        Returns:
            dict
    """
    
    audios = [btch['audio'] for btch in batch]
    audios = pad_sequence(audios, batch_first=True).view(len(batch), -1)
    
    audio_lens = torch.tensor([btch['audio_len'] for btch in batch], dtype=torch.long)
    
    texts = [btch['text'] for btch in batch]
    
    text_lens = torch.tensor([btch['text_len'] for btch in batch], dtype=torch.long)
    
    tokens = [btch['tokens'] for btch in batch]
    tokens = pad_sequence(tokens, batch_first=True).view(len(batch), -1)
    
    return {'audios': audios, # torch tensor, (batch_size, max_num_timesteps)
            'audio_lens': audio_lens, # torch tensor, (batch_size)
            'texts': texts,  # list, len=(batch_size)
            "text_lens": text_lens, # torch tensor, (batch_size)
            'tokens': tokens,  # torch tensor, (batch_size, max_text_len)
           }

In [26]:
! pytest tests/test_collate_fn.py

Don't forget to save Jupyter Notebook! 


============================= test session starts ==============================
platform linux -- Python 3.6.9, pytest-6.2.2, py-1.10.0, pluggy-0.13.1
rootdir: /home/olya/sirius-stt, configfile: pytest.ini
collected 1 item                                                               

tests/test_collate_fn.py .                                               [100%]

============================== 1 passed in 1.86s ===============================


In [27]:
dataset_train = AudioDataset(train_dataset, vocab)
dataset_val = AudioDataset(val_dataset, vocab)

In [28]:
batch_size = 90
num_workers = 8

При создании DataLoader не забудьте использовать параметры `batch_size`, `num_workers`.

Так же в DataLoader есть параметр `shuffle`, который используется для перемешивания данных. Сейчас наши аудио отсортированы по длине, т.е. длины аудио внутри одного батча максимально близки друг другу - и это самый эффективный способ формировать батчи в распознавании речи. Если включить shuffle=True, то короткие аудио могут попасть в один батч с длинными, средний размер батча увеличится, это будет менее эффективно. Поэтому необходимо **перемешивать сформированные батчи**, а не элементы датасета.
    
Реализовать перемешивание батчей в PyTorch - это не самая простая задача, поэтому советую сейчас пропустить этот шаг.

In [29]:
train_dataloader = torch.utils.data.DataLoader(
    dataset=dataset_train,
    batch_size=batch_size,
    num_workers=num_workers,
    collate_fn=collate_fn,
)
val_dataloader = torch.utils.data.DataLoader(
    dataset=dataset_val,
    batch_size=batch_size,
    num_workers=num_workers,
    collate_fn=collate_fn,
)

Давайте посмотрим, что у нас хранится в даталоадере. Обратите внимание, что из-за `batch_size=90` выведется достаточно много значений. Можно уменьшить `batch_size`, чтобы посмотреть на выход, но обучать сеть лучше с `batch_size=90`.

In [ ]:
for i, element in enumerate(train_dataloader):
    print(f'element number: {i}')
    pprint(element)
    print()
    if i > 0:
        break

# Акустические фичи

Как мы обсуждали на лекциях, есть разные способы построить аудио фичи. Мы будем использовать log mel spectrogram.


In [30]:
def compute_log_mel_spectrogram(
        audio,
        sequence_lengths,
        sample_rate=8000,
        window_size=0.02,
        window_step=0.01,
        f_min=20,
        f_max=3800,
        n_mels=64,
        window_fn=torch.hamming_window,
        power=1.0,
        eps=1e-6,
    ):
    """ Compute log-mel spectrogram.
        Input shape:
            audio: 3D tensor with shape (batch_size, num_timesteps)
            sequence_lengths: 1D tensor with shape (batch_size)
        Returns:
            4 D tensor with shape (batch_size, n_mels, new_num_timesteps)
            1D tensor with shape (batch_size)
    """

    win_length = int(window_size * sample_rate) # n_fft -- fourier
    hop_length = int(window_step * sample_rate)
    #print(f'win_length = {win_length}')
    #print(f'hop_length = {hop_length}')
    
    mel_spectrogram = torchaudio.transforms.MelSpectrogram(
        sample_rate=sample_rate,
        n_fft=win_length,
        win_length=win_length,
        hop_length=hop_length,
        f_min=f_min,
        f_max=f_max,
        n_mels=n_mels,
        window_fn=window_fn,
        power=1.0,
    )
    
    log_mel_spectrogram = torch.log(mel_spectrogram(audio) + eps) # can be problems if log(0)
    
    '''
    log_mel_spectrogram has new len because we go through the audio with window
    new len is number of windows
    '''
        
    sequence_lengths = ((sequence_lengths + 2 * hop_length - win_length) // hop_length + 1).long()
    
    return log_mel_spectrogram, sequence_lengths

In [ ]:
#! pytest tests/test_compute_log_mel_spectrogram.py

Давайте посмотрим, как выглядит log-mel спектрограмма.

In [31]:
import matplotlib.pyplot as plt

In [ ]:
'''
audio, audio_len = open_audio('test_files/test_audio.opus', 8000)
spectrogram, new_len = compute_log_mel_spectrogram(audio, audio_len)
plt.pcolormesh(spectrogram)
plt.xlabel('T')
plt.ylabel('mels')
plt.show()
Audio(data=audio.numpy(), rate=sample_rate)
'''

# Нейронная сеть

Мы подготовили все данные, теперь можно заняться реализацией нейронной сети. Будем реализовывать архитектуру [Deepspeech 2](https://arxiv.org/pdf/1512.02595.pdf) в немного упрощенном виде.


<img src="images/cat_reproduction.jpg" width="400" height="400">


Вот так будет выглядеть архитектура сети:


<img src="images/deepspeech.jpg" width="200" height="150">

In [32]:
import torch.nn as nn
import torch.nn.functional as F

In [33]:
class Model(nn.Module):
    def __init__(self, num_mel_bins, hidden_size, num_layers, num_tokens):
        super(Model, self).__init__()

        conv1_params = AttrDict(
            {
                "num_filters": 32,
                "kernel_size": [21, 11],
                "stride": [1, 1] 
            })                             
        conv2_params = AttrDict(
            {
                "num_filters": 64,
                "kernel_size": [11, 11],
                "stride": [1, 3]
            }
        )
        
        self.conv1_params = conv1_params
        self.conv2_params = conv2_params

        self.conv = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=conv1_params.num_filters,
                kernel_size=conv1_params.kernel_size,
                stride=conv1_params.stride,
                bias=False,
            ), # CONV 1
            nn.BatchNorm2d(
                num_features=conv1_params.num_filters,
                momentum=0.9,
            ), # BATCH NORM 1
            nn.ReLU(), # RELU
            nn.Conv2d(
                in_channels=conv1_params.num_filters,
                out_channels=conv2_params.num_filters,
                kernel_size=conv2_params.kernel_size,
                stride=conv2_params.stride,
                bias=False,
            ), # CONV 2
            nn.BatchNorm2d(
                num_features=conv2_params.num_filters,
                momentum=0.9,
            ), # BATCH NORM 2
            nn.ReLU(), # RELU
        )

        rnn_input_size = (num_mel_bins - conv1_params.kernel_size[0]) // conv1_params.stride[0] + 1
        rnn_input_size = (rnn_input_size - conv2_params.kernel_size[0]) // conv2_params.stride[0] + 1
        rnn_input_size *= conv2_params.num_filters

        # 4 слоя бидир lstm
        self.lstm = nn.LSTM(
            input_size=rnn_input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            bidirectional=True,
            batch_first = True,
            )
        
        self.output_layer = nn.Linear(
            in_features=hidden_size*2,
            out_features=num_tokens,
        )

    def forward(self, inputs, seq_lens, state=None):
        """
            Input shape:
                audio: 3D tensor with shape (batch_size, num_mel_bins, num_timesteps)
                sequence_lengths: 1D tensor with shape (batch_size)
            Returns:
                3D tensor with shape (new_num_timesteps, batch_size, alphabet_len)
                1D tensor with shape (batch_size)
            """
        
        outputs = inputs.unsqueeze(1) # conv2d input should be four-dimensional
        
        outputs = self.conv(outputs)
        outputs = self.transpose_and_reshape(outputs)
        outputs, hidden = self.lstm(outputs)
        outputs = self.output_layer(outputs)
        outputs = torch.transpose(outputs, 0, 1)

        seq_lens = self.get_new_seq_lens(
            seq_lens,
            self.conv1_params.kernel_size[1], 
            self.conv1_params.stride[1],
            self.conv2_params.kernel_size[1], 
            self.conv2_params.stride[1]
        ).long()

        return F.log_softmax(outputs, dim=-1), seq_lens

    @staticmethod
    def transpose_and_reshape(inputs):
    
        """ This function will be very useful for converting the output of a convolutional layer 
            to the input of a lstm layer
            
            Input shape:
                inputs: 4D tensor with shape (batch_size, num_filters, num_features, num_timesteps)
            Returns:
                3D tensor with shape (batch_size, num_timesteps, new_num_features)
            """
            
        sizes = inputs.size()
    
        # reshape
        outputs = torch.reshape(
            inputs,
            (sizes[0], 
            sizes[1]*sizes[2], 
            sizes[3])
        )  # (batch_size, num_filters * num_features, num_timesteps)
        
        # transpose
        outputs = torch.transpose(outputs, 1, 2)  # (batch_size, num_timesteps, new_num_features)
        
        return outputs
           
    @staticmethod
    def get_new_seq_lens(seq_lens, conv1_kernel_size, conv1_stride, conv2_kernel_size, conv2_stride):
    
        """ 
        Compute sequence_lengths after convolutions
        
        """
        
        new_lens = seq_lens
        new_lens = (new_lens - conv1_kernel_size) // conv1_stride + 1
        new_lens = (new_lens - conv2_kernel_size) // conv2_stride + 1

        return new_lens

In [34]:
num_mel_bins = 64
hidden_size= 512
num_layers = 4

In [35]:
model = Model(num_mel_bins=num_mel_bins,
              hidden_size=hidden_size,
              num_layers=num_layers,
              num_tokens=num_tokens)

In [36]:
model = Model(num_mel_bins=64, hidden_size=512, num_layers=4, num_tokens=35)
outputs, seq_lens = model(torch.rand([3, 64, 1000]), torch.tensor([1000, 555, 900]))

In [ ]:
#! pytest tests/test_model.py

Инициализируем модель из чекпоинта, чтобы она обучилась быстрее.

In [37]:
def load_from_ckpt(model, ckpt_path):
    checkpoint = torch.load(ckpt_path, map_location='cpu')
    model.load_state_dict(checkpoint['model_state_dict'])

In [38]:
load_from_ckpt(model, '/home/e.chuykova/data/ckpt.pt')

Тест, чтобы проверить, что код модели корректно написан, модель правильно восстановилась из чекпоинта

In [39]:
audio, audio_len = open_audio('test_files/test_audio.mp3', sample_rate)
output = model(*compute_log_mel_spectrogram(torch.unsqueeze(audio, 0), torch.unsqueeze(torch.tensor([audio_len]), 0)))
assert torch.isclose(output[0][0][0][0], torch.tensor(-3.53916406))
assert torch.isclose(output[0][15][0][30], torch.tensor(-3.605963468))

Отправим модель на гпу.

<img src="images/cuda_is_important.jpg" width="400" height="400">


In [40]:
model.cuda(device)

Model(
  (conv): Sequential(
    (0): Conv2d(1, 32, kernel_size=(21, 11), stride=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 64, kernel_size=(11, 11), stride=(1, 3), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
    (5): ReLU()
  )
  (lstm): LSTM(2176, 512, num_layers=4, batch_first=True, bidirectional=True)
  (output_layer): Linear(in_features=1024, out_features=35, bias=True)
)

# Обучаем модельку

In [41]:
from time import time
from tqdm import tqdm_notebook as tqdm
import os

Давайте напишем несколько вспомогательных функций, которые будут нам нужны для обучения модели.

Для начала займемся метриками. Основная метрика - wer (word error rate).

Тут поможет библиотека `editdistance`.

In [42]:
import editdistance

In [43]:
def calc_wer(predicted_text, gt_text):
    """ 
    Compute wer.
        Inputs:
            predicted_text: str
            gt_text: str
        Returns:
            wer: int
    """
    
    if (len(predicted_text) != 0 and len(gt_text) == 0):
        return 1
    
    if (len(predicted_text) == 0 and len(gt_text) == 0):
        return 0
    
    predicted_text = predicted_text.split()
    gt_text = gt_text.split()
        
    #wer = editdistance.eval(
    wer = editdistance.distance(
            predicted_text, 
            gt_text,
    )
    
    wer = wer / len(gt_text)
        
    return wer


def calc_wer_for_batch(list_of_predicted_text, list_of_gt_text):
    """ Compute mean wer for batch.
            Inputs:
                list_of_predicted_text: list
                list_of_gt_text: list
            Returns:int
            
    """
    mean_wer = 0
    
    for text in range(len(list_of_predicted_text)):
        mean_wer += calc_wer(
            list_of_predicted_text[text], 
            list_of_gt_text[text]
        )
        
    mean_wer /= len(list_of_predicted_text)
    
    return mean_wer

In [44]:
wer = calc_wer('', '')
wer

0

In [45]:
! pytest tests/test_compute_wer.py

Don't forget to save Jupyter Notebook! 


============================= test session starts ==============================
platform linux -- Python 3.6.9, pytest-6.2.2, py-1.10.0, pluggy-0.13.1
rootdir: /home/olya/sirius-stt, configfile: pytest.ini
collected 7 items                                                              

tests/test_compute_wer.py .......                                        [100%]

============================== 7 passed in 1.83s ===============================


Давайте реализуем greedy decoding. 

Сначала научимся получать greedy trn из выравниваний. Можно использовать `itertools`.

Не забудьте выкинуть лишние пробелы в начале и конце полученного текста!

In [46]:
import itertools

In [47]:
def decode(alignment):
    """ Get text from alignment.
        Inputs:
            alignment: str
        Returns:
            text: srt
    """
    text = ''
    i = 1

    while i < len(alignment):
        if alignment[i - 1] == '<':
            i += 7
            continue
            
        if alignment[i] == alignment[i - 1]:
            i += 1
            continue
        else:
            text += alignment[i - 1]
            i += 1
        
    if (i - 1 < len(alignment) and alignment[i - 1] != '<'):
        text += alignment[i - 1]
        
    text = text.strip()
    
    return text

In [ ]:
decode('<blank>ка<blank>к     <blank><blank>ддддее<blank>ла')

In [ ]:
! pytest tests/test_decode.py

In [48]:
import numpy as np

Получим greedy text из выхода акустической модели (logprobs).

In [49]:
def get_prediction(logprobs, logprobs_lens, vocab):
    """ 
    Compute greedy text from loglikes.
            Input shape:
                logprobs: 3D tensor with shape (num_timesteps, batch_size, alphabet_len)
                logprobs_lens: 1D tensor with shape (batch_size)
            Returns:
                list of texts with len (batch_size)
        """
    
    logprobs = torch.transpose(logprobs, 0, 1)
    
    prediction = []
    
    argmax_result = torch.argmax(logprobs, 2)
    
    for batch in range(len(logprobs_lens)):
        pred_timestep = argmax_result[batch][:int(logprobs_lens[batch])]
        prediction.append(''.join(vocab.lookup_tokens(pred_timestep.tolist())))
    
    return prediction

In [50]:
get_prediction(torch.tensor([[[0.1, 0.5, 0.4]], [[0.9, 0., 0.1]]]), torch.tensor([2]), vocab)

['ба']

In [51]:
! pytest tests/test_get_prediction.py

Don't forget to save Jupyter Notebook! 


============================= test session starts ==============================
platform linux -- Python 3.6.9, pytest-6.2.2, py-1.10.0, pluggy-0.13.1
rootdir: /home/olya/sirius-stt, configfile: pytest.ini
collected 1 item                                                               

tests/test_get_prediction.py .                                           [100%]

============================== 1 passed in 1.85s ===============================


В этой функции надо из сырых данных, извлеченных из датасета, получить спектрограмму, прогнать через модель, посчитать средний лосс и wer для батча, list с текстами предсказанных гипотез для батча.

In [52]:
def get_model_results(model, audio, audio_lens, tokens, text, text_lens, vocab, loss_fn):
    """ 
    get mean loss, mean wer and prediction list for batch
        Returns:
            loss: int
            wer: int
            prediction: list of str
            
    """
    # spectrogram
    log_mel_spectrogram, seq_lens = compute_log_mel_spectrogram(audio, audio_lens)
    
    # model
    logprobs, logprobs_lens = model.forward(log_mel_spectrogram.cuda(), seq_lens.cuda())
    
    #prediction
    prediction = get_prediction(logprobs.detach().cpu(), logprobs_lens.detach().cpu(), vocab)
    
    #wer
    wer = calc_wer_for_batch(prediction, text)
    
    #loss
    loss = loss_fn(logprobs, tokens.cuda(), logprobs_lens.long(), text_lens.long().cuda())
    
    return loss, wer, prediction

Для удобства будем логировать метрики в [tensorboard](https://pytorch.org/docs/stable/tensorboard.html).

In [53]:
from torch.utils.tensorboard import SummaryWriter

In [54]:
class TensorboardLogger:
    def __init__(self, tensorboard_path):
        self.writer = SummaryWriter(tensorboard_path)

    def log(self, step, loss, wer, mode):
        assert mode in ('train', 'val')
        
        self.writer.add_scalar(f'loss/{mode}', loss, global_step=step)
        # add loss to tb 
        
        self.writer.add_scalar(f'wer/{mode}', wer, global_step=step)
        # add wer to tb 

    def log_text(self, step, pred_texts, gt_texts, mode):
        
        for pred_text in pred_texts:
            self.writer.add_text(f'pred/{mode}', pred_text, global_step=step)
            # add pred text to tb 
            
        for gt_text in gt_texts:
            self.writer.add_text(f'gt/{mode}', gt_text, global_step=step)
            # add gt text to tb 
      

    def close(self):
        self.writer.close()

Теперь можем собрать функции в train loop. 

In [55]:
def training(model, optimizer, loss_fn, num_epochs, train_dataloader, val_dataloader, log_every_n_batch, model_dir,
             vocab):

    logger = TensorboardLogger(model_dir)

    for epoch in range(num_epochs):

        start_time = time()
        train_loss, train_wer = 0, 0
        model.train(True)

        for iteration, batch in enumerate(tqdm(train_dataloader)):
            loss, wer, prediction = get_model_results(
                model, batch["audios"], 
                batch["audio_lens"],                           
                batch["tokens"], 
                batch["texts"],                       
                batch["text_lens"], 
                vocab, 
                loss_fn,
            )
            
            # optimizer step
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += float(loss) #loss.cpu().data.numpy()
            train_wer += wer

            step = len(train_dataloader) * epoch + iteration
            if step % log_every_n_batch == 0:
                logger.log(step, loss, wer, 'train')
                logger.log_text(step, prediction, batch["texts"], "train")

        train_loss /= len(train_dataloader)
        train_wer /= len(train_dataloader)

        val_loss, val_wer = 0, 0
        model.train(False)
        
        with torch.no_grad():
            for batch in tqdm(val_dataloader):
                loss_val, wer, prediction = get_model_results(model, batch["audios"], batch["audio_lens"],
                                                              batch["tokens"], batch["texts"],
                                                              batch["text_lens"], vocab, loss_fn)
                val_loss += float(loss) #loss_val.cpu().data.numpy()
                val_wer += wer

        val_loss /= len(val_dataloader)
        val_wer /= len(val_dataloader)

        logger.log(step, val_loss, val_wer, 'val')
        logger.log_text(step, prediction, batch["texts"], "val")

        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
        }, os.path.join(model_dir, f'epoch_{epoch}.pt'))

        print(f'\nEpoch {epoch + 1} of {num_epochs} took {time() - start_time}s, ' + \
              f'train loss: {train_loss}, val loss: {val_loss}, train wer: {train_wer}, val wer: {val_wer}')

    logger.close()
    print("Finished!")

In [56]:
model = Model(num_mel_bins=num_mel_bins,
              hidden_size=hidden_size,
              num_layers=num_layers,
              num_tokens=num_tokens-1)
load_from_ckpt(model, '/home/e.chuykova/data/ckpt.pt')
model.cuda()

Model(
  (conv): Sequential(
    (0): Conv2d(1, 32, kernel_size=(21, 11), stride=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 64, kernel_size=(11, 11), stride=(1, 3), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
    (5): ReLU()
  )
  (lstm): LSTM(2176, 512, num_layers=4, batch_first=True, bidirectional=True)
  (output_layer): Linear(in_features=1024, out_features=35, bias=True)
)

In [58]:
num_epochs = 7
model_dir = 'models/1'
log_every_n_batch = 10

In [59]:
learning_rate = 2e-4
opt = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [60]:
loss_fn = torch.nn.CTCLoss(blank=blank_index, reduction='mean')

Про ctc loss очень хорошо написано [тут](https://distill.pub/2017/ctc/). А [это](https://www.cs.toronto.edu/~graves/icml_2006.pdf) исходная статья.

Если не используете перемешивание батчей (шафл), то при подборе batch size обратите внимание, что данные отсортированы (обучение будет замедляться с увеличением длины аудио).

In [ ]:
training(model, opt, loss_fn, num_epochs, train_dataloader, val_dataloader, log_every_n_batch,
             model_dir, vocab)

In [62]:
!tensorboard --logdir models --port 1234 

2021-03-22 07:50:12.679651: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2021-03-22 07:50:12.679777: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2021-03-22 07:50:12.679806: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
E0322 07:50:14.037008 140251921311552 program.py:288] TensorBoard could not bind to port 1234, it was already in use
ERROR: TensorBoard could not bind to

In [64]:
!lsof -i:1234

/bin/sh: 1: lsof: not found


<img src="images/training.jpeg" width="400" height="400">


После того, как напишете весь код и запустите обучение вашей первой модели, примерно после 5-6 эпох качество модели достигнет 30-35% WER.

# Как можно улучшить полученные результаты

<img src="images/pronunciation_or_not.jpg" width="400" height="400">

Далее описаны несколько способов, которые могут помочь улучшить качество. Потенциальный прирост обозначим ★, чем больше звездочек, тем более хорошее улучшение качества можно ожидать.

## Beam search ★★

На лекции обсуждали, что beam search помогает достичь более хорошего качества, чем greedy декодирование.

Изучить алгоритм можно [тут](https://medium.com/corti-ai/ctc-networks-and-language-models-prefix-beam-search-explained-c11d1ee23306) и [тут](https://drive.google.com/viewerng/viewer?url=https://arxiv.org/pdf/1408.2873.pdf)

Код для beam search посмотреть [тут](https://github.com/PaddlePaddle/DeepSpeech/blob/master/decoders/decoders_deprecated.py). (Отредактировать при необходимости )

## Внешняя языковая модель ★★★


Внешняя языковая модель позволяет улучшить качество, т.к. убирает условную независимость соседних символов, которая свойственна ctc лоссу.

Этот пункт состоит из двух этапов: сначала надо обучить языковую модель, затем встроить ее в beam search.

#### Обучение

Для обучения n-gram языковой модели можно использовать фреймворк kenlm.
Документация: 
* https://kheafield.com/code/kenlm/
* https://github.com/kpu/kenlm

Модель сначала строится в формате arpa, затем ее лучше перевести в формат trie. Вызывать полученную модель можно через питон [ссылка](https://github.com/kpu/kenlm#python-module)


#### Данные

Для обучения модели конечно нужны данные :) Тут есть варианты:

1. Можно обучить маленькую языковую модель на текстах из акустических обучающих данных (из трейна!).

минусы: этих данных мало

плюсы: домен остается таким же

2. Можно взять внешние данные, например, [отсюда](http://data.statmt.org/cc-100/). (46G).

минусы: тексты из другого (произвольного) домена

плюсы: данных много

При необходимости, данные надо предобработать - привести к нижнему регистру, разделить на предложения. Убрать предложения, которые содержат символы не из русского алфавита 


#### Внедрение в beam search

В разделе про beam search есть ссылки на алгоритм.

Можно использовать аргумент `ext_scoring_func` [тут](https://github.com/PaddlePaddle/DeepSpeech/blob/master/decoders/decoders_deprecated.py#L47).

Пример скорера можно найти [тут](https://github.com/PaddlePaddle/DeepSpeech/blob/master/decoders/scorer_deprecated.py)

## Аугментации ★


Позволяют искусственно увеличь размер обучающей выборки, сделать его более разнообразным.

### Аугментации аудио 

Применяются к аудиосигналу. Аугментации обычно реализуются через [sox](https://ru.wikipedia.org/wiki/SoX_(%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B0)). [Тут](http://sox.sourceforge.net/sox.html#EFFECTS) можно посмотротреть полный список sox эффектов с описанием. 

Полный список sox эффектов, доступных в torchaudio, можно посмотреть [тут](https://github.com/pytorch/audio/issues/260).

Эффекты можно комбинировать.

Внимание!

* Аугментации надо применять очень аккуратно (!) - слишком сильные аугментации только ухудшат качество. Лучше применять аугментации с некоторой вероятностью.
* Применять **только на обучающую выборку, не на валидацию!**
* Некоторые аугментации меняют sample rate и длину аудио.
* Можно применять не открывая предварительно аудиофайл [ссылка](https://pytorch.org/audio/stable/sox_effects.html#applying-effects-on-file).


Примеры:

In [ ]:
def augment_audio(audio, sample_rate, effects):
    
    effects = [effects, ['rate', '8000']]

    augmented_audio, sample_rate = torchaudio.sox_effects.apply_effects_tensor(
        torch.unsqueeze(audio, 0),
        sample_rate=sample_rate,
        effects=effects,
        channels_first=True)
    
    return augmented_audio

In [ ]:
sample_rate = 8000
audio, audio_len = open_audio('test_files/test_audio.mp3', sample_rate)
Audio(data=audio.numpy(), rate=sample_rate)

In [ ]:
augmented_audio = augment_audio(audio, sample_rate, ['treble', '20'])
Audio(data=augmented_audio.numpy(), rate=sample_rate)

In [ ]:
augmented_audio = augment_audio(audio, sample_rate, ['bass', '20'])
Audio(data=augmented_audio.numpy(), rate=sample_rate)

In [ ]:
augmented_audio = augment_audio(audio, sample_rate, ['pitch', '400'])
Audio(data=augmented_audio.numpy(), rate=sample_rate)

In [ ]:
augmented_audio = augment_audio(audio, sample_rate, ['speed', '1.5'])
Audio(data=augmented_audio.numpy(), rate=sample_rate)

In [ ]:
augmented_audio = augment_audio(audio, sample_rate, ['tempo', '1.5'])
Audio(data=augmented_audio.numpy(), rate=sample_rate)

### Аугментации спектрограммы

[SpecAugment: A Simple Data Augmentation Method for Automatic Speech Recognition](https://arxiv.org/pdf/1904.08779.pdf)

Методов [Frequency masking](https://pytorch.org/audio/stable/transforms.html#frequencymasking) и [Time masking](https://pytorch.org/audio/stable/transforms.html#timemasking) должно быть достаточно.

# Что еще можно попробовать: 

1. Поэкспериментировать с learning rate, оптимайзером (например, взять SGD). Можно добавить [lr decay](https://pytorch.org/docs/stable/optim.html#torch.optim.lr_scheduler.ExponentialLR) ★
2. Поэкспериментировать с нейронной сетью и восттановлением из чекпоинта. Например, зафиксировать предобученные слои и дообучить остальные, потом с маленьким learning rate дообучить всю модель. ★
3. Добавить новые слои в нейронную сеть. ★★
4. Использовать больше данных. ★★★
<img src="images/more_data.jpg" width="400" height="400">

В этом случае для ускорения можно запустить распределенное обучение на нескольких gpu с помощью [horovod](https://github.com/horovod/horovod). Данные можно взять тут:

* [open_stt](https://github.com/snakers4/open_stt) (до 2.5 TB данных!)
* [Russian LibriSpeech](https://openslr.org/96/) (9 GB данных)


5\. Не использовать чекпоинт и обучить свою сеть :) ★ (потребуется больше данных!)

6\. Shallow fusion - можно обучить дополнительную языковую модель и использовать в качестве рескорера 
[ссылка](https://arxiv.org/pdf/1503.03535.pdf). ★

7\. Реализовать перемешивание батчей. ★
